In [24]:
#Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
# Generate a new set of 1,500 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs

In [26]:
# Add the latitudes and longitudes in a list.
coordinates = list(lats_lngs)

In [27]:
# import the citipy module.
from citipy import citipy

In [28]:
#Create a list for holding the names of cities.
cities = []

#Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
#If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

616

In [29]:
# Import the requests library.
import requests

#Import the API key.
from config import weather_api_key

In [9]:
# Starting URL for weather map API call.
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [10]:
#Import the datetime module from the datetime library.
from datetime import datetime

In [11]:
#Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 ==0 and i>=50):
        set_count += 1
        record_count = 1
        
    #Create endpoint url for each city.
    city_url = url + "&q=" + city
        
    #log the url,record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
        
    #Add 1 to the record count.
    record_count += 1
        
    #Run an API request for each of the cities.
    try:
         #Parse the JSON and retrieve the data.
        city_weather = requests.get(city_url).json()
        #Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description=city_weather["weather"][0]["description"]
            
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        #Rain data
        try:
            city_rain = city_weather["rain"]["3h"]
            
        except:
            city_rain = 0.00 
            
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0.00
            
            
        #Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                            "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,                                                         
                             "Current Description":city_weather_description,
                              "Rain inches (last 3hrs)":city_rain,
                                "Snow inches (last 3hrs)":city_snow})
    #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
#Indicate the data loading is complete.
print("------------------------------")
print("Data Retrieval Complete       ")
print("------------------------------")       

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | oktyabrskiy
Processing Record 2 of Set 1 | chokurdakh
Processing Record 3 of Set 1 | jamestown
Processing Record 4 of Set 1 | gemena
Processing Record 5 of Set 1 | kaitangata
Processing Record 6 of Set 1 | busselton
Processing Record 7 of Set 1 | hobart
Processing Record 8 of Set 1 | laguna
Processing Record 9 of Set 1 | anori
Processing Record 10 of Set 1 | upernavik
Processing Record 11 of Set 1 | kapaa
Processing Record 12 of Set 1 | poanas
Processing Record 13 of Set 1 | dingle
Processing Record 14 of Set 1 | myitkyina
Processing Record 15 of Set 1 | acaponeta
Processing Record 16 of Set 1 | mandan
Processing Record 17 of Set 1 | albany
Processing Record 18 of Set 1 | bandar-e torkaman
City not found. Skipping...
Processing Record 19 of Set 1 | rupert
Processing Record 20 of Set 1 | port blair
Processing Record 21 of Set 1 | yellowknife
Processing Record 22 of Set 1 | port alfred
Processing R

Processing Record 38 of Set 4 | talnakh
Processing Record 39 of Set 4 | sasvad
Processing Record 40 of Set 4 | east stroudsburg
Processing Record 41 of Set 4 | cidreira
Processing Record 42 of Set 4 | qianguo
Processing Record 43 of Set 4 | touros
Processing Record 44 of Set 4 | ostrovnoy
Processing Record 45 of Set 4 | chuy
Processing Record 46 of Set 4 | benghazi
Processing Record 47 of Set 4 | heze
Processing Record 48 of Set 4 | tiznit
Processing Record 49 of Set 4 | palabuhanratu
City not found. Skipping...
Processing Record 50 of Set 4 | multan
Processing Record 1 of Set 5 | saint-prosper
Processing Record 2 of Set 5 | torbay
Processing Record 3 of Set 5 | grand river south east
City not found. Skipping...
Processing Record 4 of Set 5 | taree
Processing Record 5 of Set 5 | ahipara
Processing Record 6 of Set 5 | la cruz
Processing Record 7 of Set 5 | ilulissat
Processing Record 8 of Set 5 | kupang
Processing Record 9 of Set 5 | olinda
Processing Record 10 of Set 5 | terrace
Proces

Processing Record 26 of Set 8 | haines junction
Processing Record 27 of Set 8 | stornoway
Processing Record 28 of Set 8 | lopukhiv
Processing Record 29 of Set 8 | nantucket
Processing Record 30 of Set 8 | lasa
Processing Record 31 of Set 8 | koslan
Processing Record 32 of Set 8 | kendari
Processing Record 33 of Set 8 | namibe
Processing Record 34 of Set 8 | faya
Processing Record 35 of Set 8 | xining
Processing Record 36 of Set 8 | sawtell
Processing Record 37 of Set 8 | kirakira
Processing Record 38 of Set 8 | misratah
Processing Record 39 of Set 8 | padang
Processing Record 40 of Set 8 | kismayo
Processing Record 41 of Set 8 | chernyshevskiy
Processing Record 42 of Set 8 | nong ki
Processing Record 43 of Set 8 | saleaula
City not found. Skipping...
Processing Record 44 of Set 8 | karistos
City not found. Skipping...
Processing Record 45 of Set 8 | cockburn town
Processing Record 46 of Set 8 | genhe
Processing Record 47 of Set 8 | poum
Processing Record 48 of Set 8 | zolotinka
City no

Processing Record 11 of Set 12 | quelimane
Processing Record 12 of Set 12 | ambon
Processing Record 13 of Set 12 | warqla
City not found. Skipping...
Processing Record 14 of Set 12 | novyy urengoy
Processing Record 15 of Set 12 | saint-francois
Processing Record 16 of Set 12 | gorodishche
Processing Record 17 of Set 12 | pandamatenga
Processing Record 18 of Set 12 | banjar
Processing Record 19 of Set 12 | ngukurr
City not found. Skipping...
Processing Record 20 of Set 12 | esmeraldas
Processing Record 21 of Set 12 | bonfim
Processing Record 22 of Set 12 | gurgan
City not found. Skipping...
Processing Record 23 of Set 12 | kazalinsk
City not found. Skipping...
Processing Record 24 of Set 12 | bria
Processing Record 25 of Set 12 | dukat
Processing Record 26 of Set 12 | conceicao do castelo
Processing Record 27 of Set 12 | bensonville
Processing Record 28 of Set 12 | callaguip
Processing Record 29 of Set 12 | mumford
Processing Record 30 of Set 12 | kailua
Processing Record 31 of Set 12 |

In [12]:
len(city_data)

552

In [13]:
# Create a city dataframe.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,Oktyabrskiy,RU,2022-07-27 03:55:03,54.4815,53.4710,66.99,86,80,7.40,broken clouds,0.0,0.0
1,Chokurdakh,RU,2022-07-27 03:55:04,70.6333,147.9167,61.90,30,39,10.94,scattered clouds,0.0,0.0
2,Jamestown,US,2022-07-27 03:55:04,42.0970,-79.2353,65.84,69,40,1.99,scattered clouds,0.0,0.0
3,Gemena,CD,2022-07-27 03:55:04,3.2500,19.7667,70.02,98,100,1.97,overcast clouds,0.0,0.0
4,Kaitangata,NZ,2022-07-27 03:55:05,-46.2817,169.8464,45.50,97,100,9.95,moderate rain,0.0,0.0
5,Busselton,AU,2022-07-27 03:55:05,-33.6500,115.3333,59.74,65,99,5.48,overcast clouds,0.0,0.0
6,Hobart,AU,2022-07-27 03:55:05,-42.8794,147.3294,57.25,63,20,12.66,few clouds,0.0,0.0
7,Laguna,US,2022-07-27 03:55:06,38.4210,-121.4238,81.61,53,0,5.99,clear sky,0.0,0.0
8,Anori,BR,2022-07-27 03:55:06,-3.7728,-61.6442,75.60,72,73,2.17,broken clouds,0.0,0.0
9,Upernavik,GL,2022-07-27 03:55:06,72.7868,-56.1549,38.03,86,93,6.60,overcast clouds,0.0,0.0


In [21]:
#Export the dataframe to csv file.
city_data_df.to_csv("Weather_Database/WeatherPy_challenge.csv", index_label="City ID")

In [23]:
# Number of Cities recorded with snowfall and rainfall.
cities_rain_snow_count = len(city_data_df[(city_data_df["Rain inches (last 3hrs)"] > 0) | (city_data_df["Rain inches (last 3hrs)"] > 0)])

cities_rain_snow_count

0